# Naive Min-Max Game Tree for Tic Tac Toe

## Programmer: Chris Tralie

In [ ]:
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

class TicTac:
    def __init__(self, dim=3):
        self.dim = dim
        self.arr = [[" " for j in range(dim)] for i in range(dim)]
        self.children = []
        
    def __repr__(self):
        s = "   " + "   ".join(["{}".format(i) for i in range(self.dim)]) + "\n"
        for i in range(self.dim):
            s += "--" + "----"*(self.dim) + "\n"
            s += "{}|".format(i)
            for j in range(self.dim):
                s += " {} |".format(self.arr[i][j])
            s += "\n"
            if i == self.dim-1:
                s += "---"*(self.dim+2)
        return s
        
    def move(self, i, j, c):
        """
        Put down a move at (i, j) with player c
        
        Return false if it's an invalid move
        """
        valid = True
        if self.arr[i][j] == " ":
            self.arr[i][j] = c
        else:
            valid = False
        return valid
    
    def clone(self):
        ret = TicTac(self.dim)
        ret.arr = [[self.arr[i][j] for j in range(self.dim)] for i in range(self.dim)]
        return ret
    
    def get_neighbs(self, c):
        """
        Get TicTac neighbors with player c making a move
        """
        neighbs = []
        for i in range(self.dim):
            for j in range(self.dim):
                if self.arr[i][j] == " ":
                    neighb = self.clone()
                    neighb.arr[i][j] = c
                    neighbs.append(neighb)
        return neighbs
        
    def won(self):
        winner = " "
        for c in ["x", "o"]:
            # Check row wins
            for i in range(self.dim):
                if sum([x == c for x in self.arr[i]]) == self.dim:
                    winner = c
            # Check col wins
            for j in range(self.dim):
                if sum([self.arr[i][j] == c for i in range(self.dim)]) == self.dim:
                    winner = c
            # Check diag wins
            if sum(self.arr[i][i] == c for i in range(self.dim)) == self.dim:
                winner = c
            # Check off diag wins
            if sum(self.arr[i][self.dim-1-i] == c for i in range(self.dim)) == self.dim:
                winner = c
        return winner
    
    def is_draw(self):
        found_space = False
        for i in range(self.dim):
            for j in range(self.dim):
                if self.arr[i][j] == " ":
                    found_space = True
        return not found_space
    
    def build_tree(self, c, depth=0, max_depth=np.inf):
        """
        c: Player that just moved
        depth: int
            Depth of tree
        """
        self.children = []
        other = {"o":"x", "x":"o"}
        res = 0
        won = self.won()
        if won == " " and self.is_draw():
            res = 0
        else:
            won = self.won()
            if won != " ":
                res = {"o":1, "x":-1}[won]
            else:
                if depth < max_depth:
                    self.children = self.get_neighbs(other[c])
                    results = []
                    for child in self.children:
                        results.append(child.build_tree(other[c], depth+1, max_depth))
                    # If we're at o, the adversary is trying to minimize
                    # If we're at x, the adversary is trying to maximize
                    res = {"o":min, "x":max}[c](results)
        self.res = res
        return res
    
    def get_coords(self, idx, depth=0):
        self.y = depth
        if len(self.children) == 0:
            idx[0] += 1
            self.x = idx[0]
        else:
            for i, c in enumerate(self.children):
                if i == len(self.children)//2:
                    self.x = idx[0]
                    idx[0] += 1
                c.get_coords(idx, depth+1)
            
            
    def draw_tree_rec(self, max_depth, depth=0):
        x1, y1 = self.x, -900*self.y
        plt.text(x1, y1, "{}\n{}".format(self.res, str(self)))
        for c in self.children:
            x2, y2 = c.x, -900*c.y
            plt.plot([x1, x2], [y1, y2])
            if depth < max_depth:
                c.draw_tree_rec(max_depth, depth+1)
        
    
    def draw_tree(self, max_depth=np.inf):
        idx = [0]
        self.get_coords(idx)
        self.draw_tree_rec(max_depth)
        plt.axis("off")
        

## Play against AI

In [ ]:
t = TicTac(3)
pidx = 0

ai = "o"
other = {"x":"o", "o":"x"}
moves = {"x":np.argmin, "o":np.argmax}
while t.won() == " " and not t.is_draw():
    ipd.clear_output()
    c = ["x", "o"][pidx%2]
    if c == ai:
        t.build_tree(other[ai])
        t = t.children[moves[ai]([n.res for n in t.children])]
    else:
        print(t)
        print("Player {} input coordinates as row,col".format(c))
        [i, j] = [int(x) for x in input().split(",")][0:2]
        while not t.move(i, j, c):
            print("Invalid move!  Try again")
            [i, j] = [int(x) for x in input().split(",")][0:2]
    pidx += 1

ipd.clear_output()
print(t)
if not t.is_draw():
    print(t.won(), "wins!!")
else:
    print("Draw...game over")

## Plot and explore tree

In [ ]:
t = TicTac(3)
t.arr = [[" ", " ", " "], [" ", "o", "x"], [" ", " ", " "]]
t.build_tree("x")
t.draw_tree(max_depth=3)